# Unit tests for PoolManager

In [1]:
import fastlane_bot.db.models as models
from fastlane_bot.db.model_managers import PoolManager, PairManager, TokenManager
from fastlane_bot.db.manager import DatabaseManager

In [2]:
# Create a ContractHelper instance
pool_manager = PoolManager()

# Print and format version and date
print(f"Version: {pool_manager.__VERSION__.format('0.0.0')}")
print(f"Date: {pool_manager.__DATE__}")

Version: 3.0.1
Date: 04-26-2023


In [3]:
pair_manager = PairManager()

# Print and format version and date
print(f"Version: {pair_manager.__VERSION__.format('0.0.0')}")
print(f"Date: {pair_manager.__DATE__.format('0.0.0')}")

Version: 3.0.1
Date: 04-26-2023


In [4]:

# test TokenManager

token_manager = TokenManager()

# Print and format version and date
print(f"Version: {token_manager.__VERSION__.format('0.0.0')}")
print(f"Date: {token_manager.__DATE__.format('0.0.0')}")

Version: 3.0.1
Date: 04-26-2023


In [5]:

# DatabaseManager

db_manager = DatabaseManager()

# Print and format version and date
print(f"Version: {db_manager.__VERSION__.format('0.0.0')}")
print(f"Date: {db_manager.__DATE__.format('0.0.0')}")

Version: 3.0.1
Date: 04-26-2023


In [6]:
# Test add_token: Test if a token can be added to the database and if the token has the correct attributes.
token_manager = TokenManager()
token0 = models.Token(address="Test",
                     symbol="Test",
                     name="Test Token",
                     key="Test",
                     decimals=18)

token_manager.create_token(token0)

db_token = token_manager.session.query(models.Token).filter_by(
    address=token0.address).first()

assert db_token is not None
assert db_token.symbol == "Test"
assert db_token.name == "Test Token"
assert db_token.decimals == 18

token1 = models.Token(address="Pair",
                      symbol="Pair",
                      name="Test Token",
                      decimals=18,
                      key="Pair")

token_manager.create_token(token1)

db_token = token_manager.session.query(models.Token).filter_by(
    address=token1.address).first()

assert db_token is not None
assert db_token.symbol == "Pair"
assert db_token.name == "Test Token"
assert db_token.decimals == 18


In [7]:
from sqlalchemy.orm import Session

# - Test initialization: Ensure that the PoolManager is initialized properly with the given session object.
pair_manager = PairManager()
pool_manager = PoolManager()
assert isinstance(pool_manager.session, Session)
assert isinstance(pair_manager.session, Session)

In [8]:
# Test add pair
pair_manager = PairManager()
pair = models.Pair(name="Test/Pair",
                   tkn0_address="Test",
                   tkn1_address="Pair",
                   tkn1_key="Pair",
                   tkn0_key="Test"
                   )


pair_manager.create_pair(pair)

db_pair = pair_manager.session.query(models.Pair).filter_by(
    name=pair.name).first()

assert db_pair is not None
assert db_pair.name == "Test/Pair"
assert db_pair.tkn0_address == "Test"
assert db_pair.tkn1_address == "Pair"


In [9]:
db_manager = DatabaseManager()


blockchain = models.Blockchain(name="TestBlockchain")

db_manager.create(blockchain)

exchange = models.Exchange(name="TestExchange",
                           blockchain_name="TestBlockchain")

db_manager.create(exchange)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(TestBlockchain) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'TestBlockchain', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
# Test add_pool: Test if a pool can be added to the database and if the pool has the correct attributes.
pool_manager = PoolManager()
pool = models.Pool(id=10000000000,
                   cid='1x',
                   pair_name="Test/Pair",
                   exchange_name="TestExchange",
                   last_updated_block=10000000000,
                   address="0x1234567890abcdef1234567890abcdef12345678",
                   fee=str(0.003))
pool_manager.create_pool(pool)

db_pool = pool_manager.session.query(models.Pool).filter_by(
    exchange_name=pool.exchange_name,
    pair_name=pool.pair_name).first()
assert db_pool is not None
assert db_pool.pair_name == "Test/Pair"
assert db_pool.exchange_name == "TestExchange"

In [ ]:
# Test get_token
token = pool_manager.get_pool(address="0x1234567890abcdef1234567890abcdef12345678")
assert pool is not None
assert pool.pair_name == "Test/Pair"
assert pool.exchange_name == "TestExchange"

In [ ]:
# Test update_pool
pool = pool_manager.get_pool(address="0x1234567890abcdef1234567890abcdef12345678")
pool_manager.update_pool({"id":pool.id, "address":"Updated Test pool"})

updated_pool = pool_manager.get_pool(id=pool.id)
assert updated_pool is not None
assert updated_pool.address == "Updated Test pool"

In [ ]:
# Test delete_pool
pool = pool_manager.get_pool(address="0x1234567890abcdef1234567890abcdef12345678")
pool_manager.delete_pool(pool)

deleted_pool = pool_manager.get_pool(address="0x1234567890abcdef1234567890abcdef12345678")
assert deleted_pool is None